In [1]:
import numpy as np
import scipy.signal as scipysig
import cvxpy as cp
import matplotlib.pyplot as plt
import pandas as pd
import os

from typing import NamedTuple, Tuple, Optional, List, Union
from cvxpy import Expression, Variable, Problem, Parameter
from cvxpy.constraints.constraint import Constraint
from numpy.typing import NDArray
from pydeepc import DeePC
from pydeepc.utils import Data, create_hankel_matrix, split_data, low_rank_matrix_approximation

### To ignore warnings about cp.ECOS not being available
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
def load_timeseries(data_dir):
    """
    Read every .xlsx in data_dir, concatenating their 'u' and 'v_meas' columns.
    Returns
    -------
    u : np.ndarray, shape (T_total,)
    v : np.ndarray, shape (T_total,)
    """
    u_list, v_list = [], []
    for fname in sorted(os.listdir(data_dir)):
        if not fname.endswith('.xlsx'):
            continue
        df = pd.read_excel(os.path.join(data_dir, fname))
        u_list.append(df['u'].values)
        v_list.append(df['v_meas'].values)
    u = np.concatenate(u_list, axis=0)
    v = np.concatenate(v_list, axis=0)
    return u, v

In [3]:
# DeePC paramters
s = 1                       # How many steps before we solve again the DeePC problem
T_INI = 50                   # Size of the initial set of data
T_list = [50, 500]          # Number of data points used to estimate the system
HORIZON = 50                # Horizon length
LAMBDA_G_REGULARIZER = 1    # g regularizer (see DeePC paper, eq. 8)
LAMBDA_Y_REGULARIZER = 1    # y regularizer (see DeePC paper, eq. 8)
LAMBDA_U_REGULARIZER = 0    # u regularizer
EXPERIMENT_HORIZON = 100    # Total number of steps

DATA_DIR   = "../dataForHankle"
CACHE_FILE = "../dataForHankle/hankel_dataset.npz"

In [ ]:
if os.path.isfile(CACHE_FILE):
    npz = np.load(CACHE_FILE)
    Up, Uf, Yp, Yf = npz['U_p'], npz['U_f'], npz['Y_p'], npz['Y_f']

KeyError: 'U_p is not a file in the archive'

In [1]:
from collections import deque

In [4]:
u, v = load_timeseries(DATA_DIR)
print(u.shape, v.shape)
u = u.reshape(-1, 1)
v = v.reshape(-1, 1)
print(u.shape, v.shape)
data = Data(u,v)
Up, Uf, Yp, Yf = split_data(data, T_INI, HORIZON, explained_variance = 0.9)
print("Shapes: U_p", Up.shape, "U_f", Uf.shape, "Y_p", Yp.shape, "Y_f", Yf.shape)


(753405,) (753405,)
(753405, 1) (753405, 1)
Shapes: U_p (50, 753306) U_f (50, 753306) Y_p (50, 753306) Y_f (50, 753306)


In [ ]:
Up, Uf, Yp, Yf = split_data(data, T_INI, HORIZON, explained_variance = 0.9)
print("Shapes: U_p", Up.shape, "U_f", Uf.shape, "Y_p", Yp.shape, "Y_f", Yf.shape)


In [ ]:
np.savez(CACHE_FILE, Up=Up, Uf=Uf, Yp=Yp, Yf=Yf)


(753405,) (753405,)
(753405, 1) (753405, 1)


In [19]:
deepc = DeePC(data, Tini = T_INI, horizon = HORIZON)

In [ ]:
print("Shapes: Up", Up.shape, "Uf", Uf.shape, "Yp", Yp.shape, "Yf", Yf.shape)

Shapes: U_p (50, 753306) U_f (50, 753306) Y_p (50, 753306) Y_f (50, 753306)
